In [0]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import time
import math
import os
from tqdm import tqdm
from nltk.tokenize import TweetTokenizer
import random
from google.colab import files

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
class Args:
    filename = 'TODO'
    model = "lstm"
    n_epochs = 10000
    print_every = 20
    hidden_size = 512
    n_layers = 2
    learning_rate = 0.01
    chunk_len = 300
    batch_size = 256
    shuffle = True
    device = 'cuda'
    cuda=True
    
args = Args()

In [0]:
df = pd.read_csv('/content/drive/My Drive/SHIFT/готовые csv/comments.csv')

In [0]:
lines = [line for line in df['text']]

In [0]:
tqdm.pandas()

In [0]:
sorted(lines, key=len)[:10]

['F', '😜', '😢', '🤔', '👍', '😶', '😒', '🤣', '😏', '😂']

In [0]:
sorted(lines, key=len, reverse=True)[:10]

['Королев: Ключ на старт. Гагарин: Понял. Пауза около 30 секунд. Королев: "Кедр", я "Заря-1", дается продувка. Гагарин: Понял вас. Пауза около 10 секунд. Королев: "Кедр", я "Заря-1". Ключ поставлен на дренаж. Гагарин: Понял вас. Я "Кедр". Королев: У нас все нормально, дренажные клапана закрылись. Гагарин: У меня все нормально. Самочувствие хорошее. Настроение бодрое. К старту готов. Прием. Пауза около 40 секунд. Слышно дыхание Гагарина. Королев: "Кедр", я "Заря-1". Отошла кабель-мачта. Все нормально. Гагарин: Понял вас, почувствовал. Прием. Слышу работы клапанов. Королев: Понял вас. Хорошо. Пауза около 20 секунд. Королев: Дается зажигание, "Кедр", я "Заря-1 ". Гагарин: Понял вас, дается зажигание. Королев: Предварительная ступень. Гагарин: Понял. Королев: Промежуточная. Гагарин: Понял. Королев: Полный подъем. Гагарин: Поехали. Шум двигателей. Пауза 20 секунд. Помехи и шумы. Голоса сильно искажены. Гагарин: "Заря-1", я "Кедр". Все проходит нормально. Шум в кабине слабый. Самочувствие хо

In [0]:
tokenizer = TweetTokenizer()
lines = [' '.join(tokenizer.tokenize(line.lower())) for line in lines]

In [0]:
lines[:5]

['по моему учёные это замоскировавшиеся писатели фантасты , камуфляж у них мастерский ... всякие физики , космологии и прочие " фундаментальные " науки .',
 'черную дыру в бюджете , там по круче космической будет дыра ) ) )',
 'работает кран балка , а касок нет ! работники стоят слишком близко , при перемещении грузов при высоте подъема до 10 метров нужно как минимум на 4 метра отойти в сторону !',
 '15 лет ... : с',
 'упражнения и добавки тестостерона — ключ к импотенции 🤣']

In [0]:
collection = '\n '.join(lines)

In [0]:
collection[:500]

'по моему учёные это замоскировавшиеся писатели фантасты , камуфляж у них мастерский ... всякие физики , космологии и прочие " фундаментальные " науки .\n черную дыру в бюджете , там по круче космической будет дыра ) ) )\n работает кран балка , а касок нет ! работники стоят слишком близко , при перемещении грузов при высоте подъема до 10 метров нужно как минимум на 4 метра отойти в сторону !\n 15 лет ... : с\n упражнения и добавки тестостерона — ключ к импотенции 🤣\n искусственную гравитацию нужно уже'

In [0]:
file_len = len(collection)

In [0]:
all_characters = list(set(collection))
n_characters = len(all_characters)

In [0]:
all_characters[:10]

['😷', '🚆', '🌚', 'ー', '=', '一', '🐾', '⛽', '≈', '》']

In [0]:
n_characters

606

In [0]:
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, device, model="lstm", n_layers=1):
        super(CharRNN, self).__init__()
        self.model = model.lower()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.device = device

        self.encoder = nn.Embedding(input_size, hidden_size)
        if self.model == "gru":
            self.rnn = nn.GRU(hidden_size, hidden_size, n_layers, bidirectional=False) 
        elif self.model == "lstm":
            self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers, bidirectional=False)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        batch_size = input.size(0)
        encoded = self.encoder(input)
        output, hidden = self.rnn(encoded.view(1, batch_size, -1), hidden)
        output = self.decoder(output.view(batch_size, -1))
        return output, hidden

    def init_hidden(self, batch_size):
        if self.model == "lstm":
            return (torch.zeros(self.n_layers, batch_size, self.hidden_size).to(self.device),
                    torch.zeros(self.n_layers, batch_size, self.hidden_size).to(self.device))
        return torch.zeros(self.n_layers, batch_size, self.hidden_size).to(self.device)

In [0]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = all_characters.index(string[c])
        except:
            continue
    return tensor

def random_training_set(chunk_len, batch_size):
    inp = torch.LongTensor(batch_size, chunk_len)
    target = torch.LongTensor(batch_size, chunk_len)
    for bi in range(batch_size):
        start_index = random.randint(0, file_len - chunk_len)
        end_index = start_index + chunk_len + 1
        chunk = collection[start_index:end_index]
        inp[bi] = char_tensor(chunk[:-1])
        target[bi] = char_tensor(chunk[1:])
        inp = inp.to(device)
        target = target.to(device)
    return inp, target

def train(inp, target):
    hidden = decoder.init_hidden(args.batch_size)
    decoder.zero_grad()
    loss = 0
    for c in range(args.chunk_len):
        output, hidden = decoder(inp[:,c], hidden)
        loss += criterion(output.view(args.batch_size, -1), target[:,c])
    loss.backward()
    decoder_optimizer.step()

    return loss.data.item() / args.chunk_len

def save():
    torch.save(decoder, 'shift.pt')

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def generate(decoder, prime_str='A', predict_len=100, temperature=0.8, cuda=False):
    hidden = decoder.init_hidden(1)
    prime_str = 'космос'
    prime_input = char_tensor(prime_str).unsqueeze(0).to(device)
    predicted = prime_str

    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[:,p], hidden)
        
    inp = prime_input[:,-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]

        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char).unsqueeze(0)
        if cuda:
            inp = inp.cuda()

    return predicted

In [0]:
torch.cuda.set_device(0)
device = torch.device("cuda:0")

In [0]:
decoder = CharRNN(
    n_characters,
    128,
    n_characters,
    device,
    model='lstm',
    n_layers=2,    
).to(device)

In [0]:
decoder_optimizer = torch.optim.Adam(decoder.parameters())
criterion = nn.CrossEntropyLoss()

In [0]:
start = time.time()
all_losses = []
loss_avg = 0

try:
    print("Training for %d epochs..." % args.n_epochs)
    for epoch in tqdm(range(1, args.n_epochs + 1)):
        loss = train(*random_training_set(args.chunk_len, args.batch_size))
        loss_avg += loss

        if epoch % args.print_every == 0:
            print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / args.n_epochs * 100, loss))
            print(generate(decoder, 'IV', 100, cuda=args.cuda), '\n')
        if epoch % 50 == 0:
            save()

    print("Saving...")
    save()

except KeyboardInterrupt:
    print("Saving before quit...")
    save()


  0%|          | 0/10000 [00:00<?, ?it/s]

Training for 10000 epochs...



  0%|          | 20/10000 [00:45<6:17:36,  2.27s/it]

 45s (20 0%) 3.6516]
космос и  ж е е т у   с👣i о    ки ет в с т  звь2 н》к р нтре еул  р  .   тм    а и т р и ггмв в еомен   о   




  0%|          | 40/10000 [01:30<6:15:29,  2.26s/it]

...

 37s (9960 99%) 1.4198]
космос спутник , в принцип злетнострые ответы в космосе ? кроме подрлодили проще не спросит !
 [ id2444658 




100%|█████████▉| 9980/10000 [6:06:21<00:44,  2.20s/it]

 21s (9980 99%) 1.3955]
космоса в белым делом метеорита .
 вам даже не значит от выстороны . и американцами .
 [ id86078835 | ники 




100%|██████████| 10000/10000 [6:07:05<00:00,  2.26s/it]


 5s (10000 100%) 1.4095]
космосе . 2 . это же время шикаринка .
 [ id162616009 | рома ] , мне обидкой само не будет выдержков стоит 

Saving...


In [0]:
files.download('shift.pt')